In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# import seaborn as sns

from qolmat.imputations.imputers import ImputerMICE, ImputerRegressor
from qolmat.imputations import estimators
from qolmat.benchmark.metrics import get_metric
from qolmat.benchmark import missing_patterns
from qolmat.benchmark import comparator
from qolmat.utils import plot
from qolmat.imputations import imputers

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingRegressor, HistGradientBoostingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

from sklearn.base import (
    BaseEstimator,
    ClassifierMixin,
    RegressorMixin,
)
from sklearn.compose import make_column_selector as selector

from category_encoders.one_hot import OneHotEncoder

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [22]:
df = pd.read_csv("../data/titanic.csv", sep=";")

In [34]:
y = df["survived"] == 1

In [26]:
df = df.dropna(how="all")
df = df.drop(columns=["pclass", "survived", "name", "home.dest", "cabin", "ticket", "boat", "body"])

In [27]:
df["age"] = pd.to_numeric(df["age"], errors="coerce")
df["fare"] = pd.to_numeric(df["fare"].str.replace(",", ""), errors="coerce")

In [43]:
df.head(3)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest,title
0,1.0,1.0,"Allen, Miss. Elisabeth Walton",female,29.0,0.0,0.0,24160,2113375.0,B5,S,2,NaN,"St Louis, MO",Miss
1,1.0,1.0,"Allison, Master. Hudson Trevor",male,NaN,1.0,2.0,113781,1515500.0,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON",Master
2,1.0,0.0,"Allison, Miss. Helen Loraine",female,2.0,1.0,2.0,113781,1515500.0,C22 C26,S,NaN,NaN,"Montreal, PQ / Chesterville, ON",Miss


In [12]:
selector_cat = selector(dtype_exclude=np.number)
cols_cat = selector_cat(df)

In [13]:
imputer_rpca = imputers.ImputerRpcaNoisy()
wrapper = OneHotEncoder(handle_unknown="ignore", handle_missing="return_nan", use_cat_names=True, cols=cols_cat)
imputer_rpca = estimators.WrapperTransformer(imputer_rpca, [wrapper])

In [14]:
pipestimator = estimators.make_robust_MixteHGB(allow_new=False)
imputer_hgb = ImputerRegressor(estimator=pipestimator, handler_nan="none")

In [15]:
imputer_simple = imputers.ImputerSimple()

In [16]:
dict_imputers = {"Simple": imputer_simple, "HGB": imputer_hgb, "RPCA": imputer_rpca}
cols_to_impute = df.columns
ratio_masked = .1
generator_holes = missing_patterns.UniformHoleGenerator(n_splits=2, subset=cols_to_impute, ratio_masked=ratio_masked, sample_proportional=False)
# metrics = ["mae", "wmape", "KL_columnwise", "frechet"]
metrics = ["mae"]

In [17]:
comparison = comparator.Comparator(
    dict_imputers,
    cols_to_impute,
    generator_holes = generator_holes,
    metrics=metrics,
    max_evals=2,
)
results = comparison.compare(df)
results.style.highlight_min(color="lightgreen", axis=1)

Tested model: ImputerSimple
Tested model: ImputerRegressor


KeyboardInterrupt: 